<a href="https://colab.research.google.com/github/timsetsfire/odsc-ml-drum/blob/main/Colab%20Version%20DRUM%20Model%20Serving%20Made%20Easy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DRUM - Automated Model Serving Made Easy

 We'll get our hands dirty by 

* Building a simple regression model using Scikit
* Using DRUM for Batch Scoring
* Using DRUM to get a REST API endpoint
* Show a simple example app connected to the REST API
* H2O, Keras, XGBoost, and DataRobot
* Add a DataRobot remote agent if you are interested in further model monitoring


## Build a Model

In [1]:
!git clone https://github.com/timsetsfire/odsc-ml-drum.git

Cloning into 'odsc-ml-drum'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 345 (delta 80), reused 76 (delta 27), pack-reused 172
Receiving objects: 100% (345/345), 77.41 MiB | 37.13 MiB/s, done.
Resolving deltas: 100% (147/147), done.


In [2]:
!cat /content/odsc-ml-drum/colab-requirements.txt | sed -e '/^\s*#.*$/d' -e '/^\s*$/d' | xargs -n 1 pip install

     |████████████████████████████████| 276kB 5.7MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: attrs 20.2.0
    Uninstalling attrs-20.2.0:
      Successfully uninstalled attrs-20.2.0
     |████████████████████████████████| 8.7MB 5.1MB/s 
     |████████████████████████████████| 204kB 50.8MB/s 
     |████████████████████████████████| 788kB 49.2MB/s 
     |████████████████████████████████| 153kB 52.3MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
     |████████████████████████████████| 808kB 50.3MB/s 
     |████████████████████████████████| 204kB 51.3MB/s 
     |████████████████████████████████| 112kB 58.3MB/s 
     |████████████████████████████████| 552kB 53.4MB/s 
  Created wheel for progress: filename=progress-1.5-cp36-none-any.whl size=8074 sha256=a97910c2f92839fde6539959249220fbae95cb6f64be5554f95159bd9225502a
  Stored i

In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import pickle
import datetime

## load data

df = pd.read_csv('/content/odsc-ml-drum/data/boston_housing.csv')
df.head()

## set features and target

X = df.drop('MEDV', axis=1)
y = df['MEDV']

## train the model
rf = RandomForestRegressor(n_estimators = 20)
rf.fit(X,y)

## serialize the model

with open('/content/odsc-ml-drum/src/custom_model/rf.pkl', 'wb') as pkl:
    pickle.dump(rf, pkl)

# Batch Scoring with DRUM
<a id="setup_complete"></a>

At this point our model has been written to disk and we want to start making predictions with it.  To do this, we'll leverage DRUM and it's ability to natively handle our scikit learn model, all we need to do is tell DRUM where it resides as well as the data we wish to score.  

There are a lot of frameworks which DRUM supports nateively, but for those which DRUM doesn't support of these shelf, we'll just need to create some custom hooks so DRUM.  In this example, we'll highlight some very simple custom hooks, and will provide links to more complex examples.  

In [8]:
!drum score --code-dir /content/odsc-ml-drum/src/custom_model --input /content/odsc-ml-drum/data/boston_housing_inference.csv --output /content/odsc-ml-drum/data/predictions.csv --target-type regression

/usr/local/lib/python3.6/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
2020-10-29 12:07:41.184675: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [10]:
pd.read_csv("/content/odsc-ml-drum/data/predictions.csv").head()

,Predictions
0,27.610
1,21.630
2,34.870
3,33.765
4,35.265


# Start the inference server locally

Batch scoring can be very useful, but the utility DRUM offers does not stop there.  We can also leverage DRUM to serve our model as a RESTful API endpoint.  The only thing that changes is the way we will structure the command - using the `server` mode instead of `score` model.  We'll also need to provide an address which is NOT in use.  

When starting the server, we'll use `subprocess.Popen` so we may interact with the server in this notebook

In [11]:
import subprocess
import requests
import pandas as pd
from io import BytesIO
import yaml
import time
import os
import datarobot as dr
from pprint import pprint

In [12]:
run_inference_server = ["drum",
              "server",
              "--code-dir","/content/odsc-ml-drum/src/custom_model", 
              "--address", "0.0.0.0:6789", 
              "--show-perf",
              "--target-type", "regression",
              "--logging-level", "info",
              "--show-stacktrace",
#               "--verbose"
              ]

In [13]:
inference_server = subprocess.Popen(run_inference_server, stdout=subprocess.PIPE)

## Ping the Server to make sure it is running

In [14]:
## confirm the server is running
time.sleep(5) ## snoozing before pinging the server to give it time to actually start
print('check status')
requests.request("GET", "http://0.0.0.0:6789/").content

check status


b'{"message":"OK"}\n'

## Send data to server for inference

The request must provide our dataset as form data.  In order to do so, we'll create a simple python function to pass the data over appropriately.  We'll leverage the same function in our simple flask app a little later.  

In [15]:
def score(data, port = "6789"):
    b_buf = BytesIO()
    b_buf.write(data.to_csv(index=False).encode("utf-8"))
    b_buf.seek(0)
  
    url = "http://localhost:{}/predict/".format(port)
    files = [
        ('X', b_buf)
    ]
    response = requests.request("POST", url, files = files, timeout=None, verify=False)
    return response

In [16]:
# %%timeit
scoring_data = pd.read_csv("/content/odsc-ml-drum/data/boston_housing_inference.csv")
predictions = score(scoring_data).json() ## score entire dataset but only show first 5 records
pprint(predictions)

{'predictions': [27.61,
                 21.63,
                 34.87,
                 33.765,
                 35.265,
                 26.595,
                 22.51,
                 23.465,
                 16.235]}


## Start the Flask App

Now that we testing out our endpoing, now it is time to start up our flask app, but first, we need to set a few environment variables for the flask app.  

In [17]:
# os.environ["LC_ALL"] = "C.UTF-8"
# os.environ["LANG"] = "C.UTF-8"
# os.environ["FLASK_APP"] = "server.app"
# os.environ["FLASK_ENV"] = "development"

Running the flask app as follows will lock the interpreter and only return control once you interupt the kernal.  Be advised that interupting the kernel via the `stop` button will kill the flask app AND the inference server.  Once you execute the following cell, go over to the [app](http://localhost:8080/frontend)

In [ ]:
# !cd /content/odsc-ml-drum/src && python -m flask run --host 0.0.0.0 --port 8080

 * Serving Flask app "server.app" (lazy loading)
 * Environment: development
 * Debug mode: on
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 208-528-333
127.0.0.1 - - [27/Oct/2020 11:19:23] "GET /frontend HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2020 11:19:23] "GET /static/styles/layout.css HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2020 11:19:23] "GET /favicon.ico HTTP/1.1" 404 -
      CRIM    ZN  INDUS  CHAS    NOX  ...  RAD    TAX  PTRATIO      B  LSTAT
0  0.00632  18.0   2.31   0.0  0.538  ...  1.0  296.0     15.3  396.9   4.98

[1 rows x 13 columns]
making request
prediciton [25.725]
heylksdfmlsdmsdflklmsdfsdf
127.0.0.1 - - [27/Oct/2020 11:19:25] "POST /frontend HTTP/1.1" 200 -
^C


In [18]:
# inference_server.terminate()
# inference_server.stdout.readlines()

In [40]:
requests.request("GET", "http://0.0.0.0:6789/").content

b'{"message":"OK"}\n'

In [41]:
requests.request("POST", "http://0.0.0.0:6789/shutdown/").content

b'Server shutting down...'

## Value Prop

One may ask, what is the benefit to be had here?  Well, first of, there is not need for me to write an api to get the model up and running.  Second, DRUM allows me to abstract the framework away (provided I'm using one that is natively supported, or I can write enough python so that DRUM understands how to hook up to the model.  

For example, I could hot swap models as I see fit (see exampels in `./src/other_models`)

While we will run through several other frameworks with in `score` you can bet they are supported in `server` mode as well!

#### H2O Mojo

In [20]:
!drum score --code-dir /content/odsc-ml-drum/src/other_models/h2o_mojo/regression --input /content/odsc-ml-drum/data/boston_housing_inference.csv --target-type regression


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
   Predictions
0    24.504000
1    22.492000
2    34.554001
3    34.420001
4    35.289001
5    28.394001
6    21.936000
7    23.451000
8    17.065000


#### Keras

In [21]:
!drum score --code-dir /content/odsc-ml-drum/src/other_models/python3_keras_joblib --input /content/odsc-ml-drum/data/boston_housing_inference.csv --target-type regression


2020-10-29 12:11:07.836053: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying t

#### XGBoost

Requires XGBoost

In [22]:
!drum score --code-dir /content/odsc-ml-drum/src/other_models/python3_xgboost --input /content/odsc-ml-drum/data/boston_housing_inference.csv --target-type regression


/usr/local/lib/python3.6/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
2020-10-29 12:11:29.978576: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
   Predictions
0    24.541843
1    21.260277
2    34.018497
3    32.569200
4    34.248066
5    27.282364
6    20.803959
7    19.645220
8    16.968880


#### DataRobot Codegen

In [23]:
!drum score --code-dir /content/odsc-ml-drum/src/other_models/dr_codegen --input /content/odsc-ml-drum/data/boston_housing_inference.csv --target-type regression


   Predictions
0    24.258228
1    24.258228
2    32.451515
3    32.451515
4    32.451515
5    24.258228
6    21.078378
7    13.107812
8    13.107812


# Monitoring Deployments

What follows will require a DataRobot account.  You can get a trial account at [https://www.datarobot.com/trial/](https://www.datarobot.com/trial/)

Also, JDK 11 or 12 will be required.

The main idea: we'll will start an agent service locally.  This agent will be monitoring a spooler.  The spooler could be something as simple as local file system, or a little more realistic like a message broker (pubsub, rabbitmq, sqs).  

Once, this agent is spun up locally, we'll enable a few environment variables to let DRUM know that there is an agent present and that it needs to buffer data to defined spool.  

## Getting the monitoring agents



Currently - have to go in through the [UI](https://app2.datarobot.com/account/developer-tools) to grab the agents 

In [25]:
token = "NWY2Y2RlMDhkMDgzMGIwMzA2NjU0ZjUwOkhrVnVsMS9oTmt6REI1N3BoSG5NV0hzMUxRaFFzTXhTYUYrVVNWY3BySG89"
endpoint = "https://app2.datarobot.com"
## connect to DataRobot platform with python client. 
client = dr.Client(token, "{}/api/v2".format(endpoint))
# mlops_agents_tb = client.get("mlopsInstaller")
# with open("/content/odsc-ml-drum/mlops-agent.tar.gz", "wb") as f:
#     f.write(mlops_agents_tb.content)

In [26]:
# !tar -xf /content/mlops-agent.tar.gz -C ..
!tar -xf /content/datarobot-mlops-agent-6.2.4-399.tar.gz -C .

## Configuring the Agent

When we'll configure the agent, we just need to define the DataRobot MLOPS location, our api token.  By default, the agent will expect the data to be spooled on the local file system.  Specifically, the default location will be `/tmp/ta` so we just need to make sure that location exists

In [28]:
!mkdir -p /tmp/ta

In [27]:
agents_dir = "/content/datarobot-mlops-agent-6.2.4"
with open(r'{}/conf/mlops.agent.conf.yaml'.format(agents_dir)) as file:
    documents = yaml.load(file, Loader=yaml.FullLoader)
## configure the loaction of the mlops instance with which we'll communcate
documents['mlopsUrl'] = endpoint
# Set your API token
documents['apiToken'] = token
## write the configuration back to disk
with open('../{}/conf/mlops.agent.conf.yaml'.format(agents_dir), "w") as f:
    yaml.dump(documents, f)

## Start the Agent Service

Checking to make sure we can start up the agents service.  

This will require a JDK - tested with 11 and 12

In [29]:
## run agents service
subprocess.call("{}/bin/start-agent.sh".format(agents_dir))

0

In [30]:
## check status
check = subprocess.Popen(["../{}/bin/status-agent.sh".format(agents_dir)], stdout=subprocess.PIPE)
print(check.stdout.readlines())
check.terminate()

[b'DataRobot MLOps-Agent is running as a service.\n']


In [31]:
## check log to see that the agent connected to DR MLOps
check = subprocess.Popen(["cat", "../{}/logs/mlops.agent.log".format(agents_dir)], stdout=subprocess.PIPE)
for line in check.stdout.readlines():
    print(line)
check.terminate()

b'2020-10-29 12:17:23,828 INFO  com.datarobot.mlops.agent.config.channels.YamlBuilder        [] - Found spooler of type FILESYSTEM\n'
b'2020-10-29 12:17:23,831 INFO  com.datarobot.mlops.agent.config.channels.YamlBuilder        [] - Setting directory = /tmp/ta\n'
b'2020-10-29 12:17:23,831 INFO  com.datarobot.mlops.agent.config.channels.YamlBuilder        [] - Setting CHANNEL_NAME = filesystem\n'
b"2020-10-29 12:17:24,858 INFO  com.datarobot.mlops.common.client.MLOpsClient                [] - DataRobot Server API Version found: '2.22'\n"
b"2020-10-29 12:17:25,042 INFO  com.datarobot.mlops.common.client.MLOpsClient                [] - DataRobot Server API Version found: '2.22'\n"
b"2020-10-29 12:17:25,042 INFO  com.datarobot.mlops.agent.Agent                              [] - DataRobot server at 'https://app2.datarobot.com' is reachable.\n"
b'2020-10-29 12:17:25,043 INFO  com.datarobot.mlops.agent.Agent                              [] - DataRobot Monitoring Agent will process 100 records 

## DataRobot MLOps - Deploying External Model 

To communication with DataRobot MLOps, with need to MLOps python client installed which came in the downloaded tarball

In [32]:
!pip install /content/datarobot-mlops-*/lib/datarobot_mlops-*.whl -q

     |████████████████████████████████| 133kB 5.3MB/s 
     |████████████████████████████████| 5.9MB 13.9MB/s 
     |████████████████████████████████| 112kB 54.6MB/s 
     |████████████████████████████████| 204kB 53.3MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 552kB 51.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [33]:
from datarobot.mlops.mlops import MLOps
from datarobot.mlops.common.enums import OutputType
from datarobot.mlops.connected.client import MLOpsClient
from datarobot.mlops.common.exception import DRConnectedException
from datarobot.mlops.constants import Constants

In [34]:
DEPLOYMENT_NAME="Boston Housing Prices ODSC"
TRAINING_DATA = '/content/odsc-ml-drum/data/boston_housing.csv'

In [35]:
model_info = {
        "name": "Boston Housing Pricins",
        "modelDescription": {
            "description": "prediction price of home"
        },
        "target": {
            "type": "Regression",
            "name": "MEDV",
        }
}

In [36]:
# Create connected client
mlops_client = MLOpsClient(endpoint, token)

# Add training_data to model configuration
print("Uploading training data - {}. This may take some time...".format(TRAINING_DATA))
dataset_id = mlops_client.upload_dataset(TRAINING_DATA)
print("Training dataset uploaded. Catalog ID {}.".format(dataset_id))
model_info["datasets"] = {"trainingDataCatalogId": dataset_id}

# Create the model package
print('Create model package')
model_pkg_id = mlops_client.create_model_package(model_info)
model_pkg = mlops_client.get_model_package(model_pkg_id)
model_id = model_pkg["modelId"]

# Deploy the model package
print('Deploy model package')
deployment_id = mlops_client.deploy_model_package(model_pkg["id"],
                                                            DEPLOYMENT_NAME)

# Enable data drift tracking
print('Enable feature drift')
enable_feature_drift = TRAINING_DATA is not None
mlops_client.update_deployment_settings(deployment_id, target_drift=True,
                                                  feature_drift=enable_feature_drift)
_ = mlops_client.get_deployment_settings(deployment_id)

print("\nDone.")
print("DEPLOYMENT_ID=%s, MODEL_ID=%s" % (deployment_id, model_id))

DEPLOYMENT_ID = deployment_id
MODEL_ID = model_id

Uploading training data - /content/odsc-ml-drum/data/boston_housing.csv. This may take some time...
Training dataset uploaded. Catalog ID 5f9ab30deb01f601685bd4c4.
Create model package
Deploy model package
Enable feature drift

Done.
DEPLOYMENT_ID=5f9ab330af091a01778bf76c, MODEL_ID=5f9ab32f8147d603e43e4405


In [37]:
from IPython.core.display import display, HTML
link = "{}/deployments/{}/overview".format(endpoint,deployment_id)
display(HTML("""<a href="{link}">{link}</a>""".format( link=link )))

# Adding Monitoring with MLOps Monitoring Agents

## Monitoring With DRUM

There are a few addition parameters we should set for the command line utility, or we may just create environment variables, and allow the drum utility to pick up the details from there.  

```
  --monitor             Monitor predictions using DataRobot MLOps. True or
                        False. (env: MONITOR).Monitoring can not be used in
                        unstructured mode.
  --deployment-id DEPLOYMENT_ID
                        Deployment id to use for monitoring model predictions
                        (env: DEPLOYMENT_ID)
  --model-id MODEL_ID   MLOps model id to use for monitoring predictions (env:
                        MODEL_ID)
  --monitor-settings MONITOR_SETTINGS
                        MLOps setting to use for connecting with the MLOps
                        Agent (env: MONITOR_SETTINGS)
```
For today, we'll set environment variables to add monitoring. 


In [38]:
os.environ["MONITOR"] = "True"
os.environ["DEPLOYMENT_ID"] = deployment_id
os.environ["MODEL_ID"] = model_id
os.environ["MONITOR_SETTINGS"] = "spooler_type=filesystem;directory=/tmp/ta;max_files=5;file_max_size=1045876000"

In [42]:
run_inference_server = ["drum",
              "server",
              "--code-dir","/content/odsc-ml-drum/src/custom_model", 
              "--address", "0.0.0.0:43210", 
              "--show-perf",
              "--target-type", "regression",
              "--logging-level", "info",
              "--show-stacktrace",
#               "--verbose"
              ]

In [43]:
inference_server_with_monitoring = subprocess.Popen(run_inference_server, stdout=subprocess.PIPE)

In [44]:
predictions = score(
    pd.read_csv("/content/odsc-ml-drum/data/boston_housing.csv").drop(["MEDV"],axis=1).head(100),
    "43210")

In [45]:
pd.DataFrame(predictions.json()).head()

,predictions
0,27.610
1,21.630
2,34.870
3,33.765
4,35.265


In [46]:
requests.post("http://localhost:43210/shutdown/").content

b'Server shutting down...'

In [47]:
subprocess.call("../{}/bin/stop-agent.sh".format(agents_dir))

0

In [48]:
## check that agent is stopped 
check = subprocess.Popen(["../{}/bin/status-agent.sh".format(agents_dir)], stdout=subprocess.PIPE)
print(check.stdout.readlines())
check.terminate()

[b'DataRobot MLOps-Agent is not running as a service.\n']


In [49]:
display(HTML("""<a href="{link}">{link}</a>""".format( link=link )))

In [50]:
deployment = dr.Deployment.get(deployment_id)
deployment.get_service_stats()

ServiceStats(5f9ab32f8147d603e43e4405 | 2020-10-22 13:00:00+00:00 - 2020-10-29 13:00:00+00:00)

In [51]:
service_stats = deployment.get_service_stats()
service_stats.metrics

{'cacheHitRatio': 0,
 'executionTime': 14.751672744751,
 'medianLoad': 0,
 'numConsumers': 1,
 'peakLoad': 1,
 'responseTime': 0,
 'serverErrorRate': 0,
 'slowRequests': 0,
 'totalPredictions': 100,
 'totalRequests': 1,
 'userErrorRate': 0}